<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C6_process_track4tracker_output_for_deepsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#config
PROJECT = 'rc_car_tracking'
MODEL = 'deepsort'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'

DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}'
DRIVE_IMAGES_FOLDER = f'{MODEL}_images'
DRIVE_DATASET_FOLDER = f'{MODEL}_dataset'
DRIVE_TRACK4TRACKER_FOLDER = f'{MODEL}_track4tracker'

COSINE_METRIC_LEARNING_CAMERA_ID = 1
COSINE_METRIC_LEARNING_TRAIN_FOLDER = 'image_train'
COSINE_METRIC_LEARNING_TEST_FOLDER = 'image_test'

TRAIN_PERCENTAGE = 0.8
WRITE_MODE = True

In [ ]:
#imports
import json
import cv2
import os
import time
import shutil
import sys
from google.colab.patches import cv2_imshow
from google.colab import drive

In [ ]:
#google drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)

In [ ]:
#symbolic link
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
# init COSINE_METRIC_LEARNING_TRAIN_FOLDER and COSINE_METRIC_LEARNING_TEST_FOLDER
shutil.rmtree(f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TRAIN_FOLDER}', ignore_errors=True)
os.makedirs(f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TRAIN_FOLDER}', exist_ok=True)

shutil.rmtree(f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TEST_FOLDER}', ignore_errors=True)
os.makedirs(f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TEST_FOLDER}', exist_ok=True)

In [ ]:
#generating crops and relative trasformations
import math

start = time.time()

temp = []

json_folder = f'{COLAB_BASE_PROJECT}/{DRIVE_TRACK4TRACKER_FOLDER}'

for i,file in enumerate(os.listdir(json_folder)):
  entity_id = i + 1
  
  try:
    f = open(os.path.join(f'{json_folder}/{file}'))
    data = json.load(f)
  except:
    print(f'Error in loading JSON... {json_folder}/{file} :(')
    sys.exit(1)

  len_train = math.ceil(len(data) * TRAIN_PERCENTAGE)

  for j, image in enumerate(data):
    path = f'{COLAB_BASE_PROJECT}/{DRIVE_IMAGES_FOLDER}/{image}'

    frame = image.split('_')[1].split('.')[0]

    if not os.path.isfile(path):
      print(f'File {path} not found...')
      sys.exit(1)

    box = data[image]
    img = cv2.imread(path)
    x = int(box[0])
    y = int(box[1])
    w = int(box[2])
    h = int(box[3])

    cropped = img[y:y+h, x:x+w]
    #flip image horizzontally
    flipped = cv2.flip(cropped, 1)
    #rotate image
    rotated = cv2.transpose(cropped)
    rotated= cv2.flip(rotated,flipCode=1)

    if WRITE_MODE:
      output_folder = f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TRAIN_FOLDER}'

      #print(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}.jpg')
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}.jpg', cropped)
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}_flipped.jpg', flipped)
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}_rotated.jpg', rotated)

      output_folder = f'{COLAB_BASE_PROJECT}/{DRIVE_DATASET_FOLDER}/{COSINE_METRIC_LEARNING_TEST_FOLDER}'
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}.jpg', cropped)
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}_flipped.jpg', flipped)
      cv2.imwrite(f'{output_folder}/{entity_id}_{COSINE_METRIC_LEARNING_CAMERA_ID}_{frame}_rotated.jpg', rotated)
    else:
      cv2_imshow(cropped)
      cv2_imshow(flipped)
      cv2_imshow(rotated)
    
    cv2.waitKey(0)

  f.close()

end = time.time()
print(f"Job finished in {end-start} seconds :)")